In [1]:
# le basics
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from time import strftime

# le mon creation
import env
import acquire as a
import prepare as p
import explore as e
from pprint import pprint

# for text digestion
import unicodedata
import re
import json
from bs4 import BeautifulSoup

# nltk: natural language toolkit -> tokenization, stopwords
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from collections import Counter

# what's cookin', good lookin'?
import warnings
warnings.filterwarnings('ignore')

# Acquire

In [2]:
df_acquire = a.acquire_readmes()

In [3]:
df = df_acquire.copy()

In [4]:
df.head()

,repo,language,readme_contents
0,microsoft/terminal,C++,![terminal-logos](https://user-images.githubus...
1,microsoft/PowerToys,C#,# Microsoft PowerToys\n\n![Hero image for Micr...
2,huggingface/transformers,Python,<!---\nCopyright 2020 The HuggingFace Team. Al...
3,rust-lang/rust,Rust,# The Rust Programming Language\n\nThis is the...
4,mtdvio/every-programmer-should-know,NaN,> *[Join our community](https://metadevelopmen...


In [5]:
df.shape

(683, 3)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 682
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             683 non-null    object
 1   language         619 non-null    object
 2   readme_contents  681 non-null    object
dtypes: object(3)
memory usage: 21.3+ KB


# Prepare

In [7]:
df_prepare = p.prep_df_for_nlp(df, 'readme_contents', extra_words = p.EXTRA_WORDS)

In [8]:
df = df_prepare.copy()

In [9]:
df.shape

(683, 6)

In [10]:
df.head()

,repo,language,readme_contents,clean,stem,lemmatized
0,microsoft/terminal,Other,![terminal-logos](https://user-images.githubus...,terminallogoshttpsuserimagesgithubusercontentc...,terminallogoshttpsuserimagesgithubusercontentc...,terminallogoshttpsuserimagesgithubusercontentc...
1,microsoft/PowerToys,Other,# Microsoft PowerToys\n\n![Hero image for Micr...,microsoft powertoys hero microsoft powertoysdo...,microsoft powertoy hero microsoft powertoysdoc...,microsoft powertoys hero microsoft powertoysdo...
2,huggingface/transformers,Python,<!---\nCopyright 2020 The HuggingFace Team. Al...,copyright 2020 huggingface team rights reserve...,copyright 2020 huggingfac team right reserv li...,copyright 2020 huggingface team right reserved...
3,rust-lang/rust,Other,# The Rust Programming Language\n\nThis is the...,rust programming language main source code rep...,rust program languag main sourc code repositor...,rust programming language main source code rep...
4,mtdvio/every-programmer-should-know,Not Listed,> *[Join our community](https://metadevelopmen...,join communityhttpsmetadevelopmentio professio...,join communityhttpsmetadevelopmentio professio...,join communityhttpsmetadevelopmentio professio...


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 682
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   repo             683 non-null    object  
 1   language         683 non-null    category
 2   readme_contents  683 non-null    object  
 3   clean            683 non-null    object  
 4   stem             683 non-null    object  
 5   lemmatized       683 non-null    object  
dtypes: category(1), object(5)
memory usage: 33.0+ KB


In [12]:
train, validate, test = p.split_data(df, 'language')

In [13]:
df = train.copy()

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443 entries, 482 to 8
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   repo             443 non-null    object  
 1   language         443 non-null    category
 2   readme_contents  443 non-null    object  
 3   clean            443 non-null    object  
 4   stem             443 non-null    object  
 5   lemmatized       443 non-null    object  
dtypes: category(1), object(5)
memory usage: 21.5+ KB


# Explore based on Ryan's notebooks

# Top five languages

### make the following cell Code to generate language series

JavaScript_words_series = (' '.join(df[df.language == 'JavaScript']['readme_contents']))
Python_words_series = (' '.join(df[df.language == 'Python']['readme_contents']))
TypeScript_words_series = (' '.join(df[df.language == 'TypeScript']['readme_contents']))
Go_words_words_series = (' '.join(df[df.language == 'Go']['readme_contents']))
Java_words_series = (' '.join(df[df.language == 'Java']['readme_contents']))

all_words_series = (' '.join(df['readme_contents']))


In [14]:
# javascript_series, python_series, type_series, go_series, java_series, unlisted, other, all_words_series
javascript_series, python_series, type_series, go_series, java_series, unlisted_series, other_lang_series, all_words_series = p.series_generator(df)



# Thursday: Freqency of a word within a single readme

In [15]:
# plan have loop run through the contenct of every read me and count the words that are in that readme. 
# ham_words = clean(' '.join(df[df.label == 'ham']['text']))
# readme_words = join ['lemmatized'] but df.label = row
df.head()
## let's get some sights on word frequency by taking our words back apart
# we will split each set of words by the spaces,
# turn that into a list, cast that list as a Series,
# and then take the value counts of that Series
# We will do this for each type of word present
#ham_freq = pd.Series(ham_words).value_counts()

,repo,language,readme_contents,clean,stem,lemmatized
482,sdras/awesome-actions,Not Listed,"<p align=""center"">\n <br>\n <img src=""awes...",p aligncenter br srcawesomeactionspng width150...,p aligncent br srcawesomeactionspng width150 b...,p aligncenter br srcawesomeactionspng width150...
646,InstaPy/InstaPy,Python,"<p align=""center"">\n <img src=""https://i.imgu...",p aligncenter srchttpsiimgurcomsjzfzsljpg widt...,p aligncent srchttpsiimgurcomsjzfzsljpg width1...,p aligncenter srchttpsiimgurcomsjzfzsljpg widt...
474,osquery/osquery,Other,"# osquery\n\n<p align=""center"">\n<img alt=""osq...",osquery p aligncenter altosquery logo width200...,osqueri p aligncent altosqueri logo width200 s...,osquery p aligncenter altosquery logo width200...
545,hzlzh/Best-App,Not Listed,\nBest App\n----\n\n*经常会有朋友想知道有哪些 Apps 或 服务 是值...,best app apps bestapp ios app httpsappsappleco...,best app app bestapp io app httpsappsapplecomc...,best app apps bestapp io app httpsappsapplecom...
539,actix/actix-web,Other,actix-web/README.md,actixwebreadmemd,actixwebreadmemd,actixwebreadmemd


In [16]:
df.lemmatized.info()

<class 'pandas.core.series.Series'>
Int64Index: 443 entries, 482 to 8
Series name: lemmatized
Non-Null Count  Dtype 
--------------  ----- 
443 non-null    object
dtypes: object(1)
memory usage: 6.9+ KB


AttributeError: 'str' object has no attribute 'to_list'

In [43]:
wnl = nltk.stem.WordNetLemmatizer()

In [53]:
lemmas = [wnl.lemmatize(word) for word in df.lemmatized[1]]

In [54]:
lemmas

['m',
 'i',
 'c',
 'r',
 'o',
 's',
 'o',
 'f',
 't',
 ' ',
 'p',
 'o',
 'w',
 'e',
 'r',
 't',
 'o',
 'y',
 's',
 ' ',
 'h',
 'e',
 'r',
 'o',
 ' ',
 'm',
 'i',
 'c',
 'r',
 'o',
 's',
 'o',
 'f',
 't',
 ' ',
 'p',
 'o',
 'w',
 'e',
 'r',
 't',
 'o',
 'y',
 's',
 'd',
 'o',
 'c',
 'i',
 'm',
 'a',
 'g',
 'e',
 's',
 'o',
 'v',
 'e',
 'r',
 'v',
 'i',
 'e',
 'w',
 'p',
 't',
 'h',
 'e',
 'r',
 'o',
 'i',
 'm',
 'a',
 'g',
 'e',
 'p',
 'n',
 'g',
 ' ',
 'u',
 's',
 'e',
 ' ',
 'p',
 'o',
 'w',
 'e',
 'r',
 't',
 'o',
 'y',
 's',
 'u',
 's',
 'i',
 'n',
 'g',
 'p',
 'o',
 'w',
 'e',
 'r',
 't',
 'o',
 'y',
 's',
 'd',
 'o',
 'c',
 's',
 'l',
 'i',
 'n',
 'k',
 ' ',
 'd',
 'o',
 'w',
 'n',
 'l',
 'o',
 'a',
 'd',
 's',
 ' ',
 'r',
 'e',
 'l',
 'e',
 'a',
 's',
 'e',
 ' ',
 'n',
 'o',
 't',
 'e',
 's',
 'g',
 'i',
 't',
 'h',
 'u',
 'b',
 'r',
 'e',
 'l',
 'e',
 'a',
 's',
 'e',
 'l',
 'i',
 'n',
 'k',
 ' ',
 'c',
 'o',
 'n',
 't',
 'r',
 'i',
 'b',
 'u',
 't',
 'i',
 'n',
 'g',
 ' ',
 'p'

In [27]:
df1 = df.lemmatized.str.split().apply(lambda x: Counter(x))

In [28]:
df1 

482    {'p': 2, 'aligncenter': 1, 'br': 2, 'srcawesom...
646    {'p': 7, 'aligncenter': 3, 'srchttpsiimgurcoms...
474    {'osquery': 12, 'p': 4, 'aligncenter': 2, 'alt...
545    {'best': 1, 'app': 12, 'apps': 9, 'bestapp': 1...
539                              {'actixwebreadmemd': 1}
                             ...                        
47     {'slyhttpdarsainsly': 1, 'javascript': 1, 'lib...
268    {'20230112': 1, '0000': 1, 'android': 1, 'wind...
230    {'english': 2, 'repo': 1, 'gitbook': 1, 'branc...
304                                      {'readmemd': 1}
8      {'github': 9, 'licensehttpsimgshieldsiogithubl...
Name: lemmatized, Length: 443, dtype: object

In [29]:
def lemm_counter(df):
    df.lemmatized.str.split().apply(lambda x: Counter(x))
    return df

In [30]:
df2 = lemm_counter(df)
df2

,repo,language,readme_contents,clean,stem,lemmatized
482,sdras/awesome-actions,Not Listed,"<p align=""center"">\n <br>\n <img src=""awes...",p aligncenter br srcawesomeactionspng width150...,p aligncent br srcawesomeactionspng width150 b...,p aligncenter br srcawesomeactionspng width150...
646,InstaPy/InstaPy,Python,"<p align=""center"">\n <img src=""https://i.imgu...",p aligncenter srchttpsiimgurcomsjzfzsljpg widt...,p aligncent srchttpsiimgurcomsjzfzsljpg width1...,p aligncenter srchttpsiimgurcomsjzfzsljpg widt...
474,osquery/osquery,Other,"# osquery\n\n<p align=""center"">\n<img alt=""osq...",osquery p aligncenter altosquery logo width200...,osqueri p aligncent altosqueri logo width200 s...,osquery p aligncenter altosquery logo width200...
545,hzlzh/Best-App,Not Listed,\nBest App\n----\n\n*经常会有朋友想知道有哪些 Apps 或 服务 是值...,best app apps bestapp ios app httpsappsappleco...,best app app bestapp io app httpsappsapplecomc...,best app apps bestapp io app httpsappsapplecom...
539,actix/actix-web,Other,actix-web/README.md,actixwebreadmemd,actixwebreadmemd,actixwebreadmemd
...,...,...,...,...,...,...
47,darsain/sly,JavaScript,# [Sly](http://darsa.in/sly)\n\nJavaScript lib...,slyhttpdarsainsly javascript library onedirect...,slyhttpdarsainsli javascript librari onedirect...,slyhttpdarsainsly javascript library onedirect...
268,freefq/free,Not Listed,更新时间 2023-01-12 00:00 \n所有免费节点都爬取自网络，请勿用于非法用途...,20230112 0000 android windows v2ray v2raynghtt...,20230112 0000 android window v2ray v2raynghttp...,20230112 0000 android window v2ray v2raynghttp...
230,geekxh/hello-algorithm,Java,## 简介\n\nEnglish version repo and Gitbook is o...,english repo gitbook english branchhttpsgithub...,english repo gitbook english branchhttpsgithub...,english repo gitbook english branchhttpsgithub...
304,stedolan/jq,Other,README.md,readmemd,readmemd,readmemd


In [62]:
print(type(df.lemmatized))

<class 'pandas.core.series.Series'>


In [ ]:
row1 = df.lemmatized

In [64]:
row1 = df.lemmatized[1]

In [31]:
ones = e.get_ngram_frequency(df.lemmatized)

In [32]:
freq_words = list(ones[ones >10].index)

In [68]:
# BEWARE THIS BOX IS REALLY LONG!!!!!

#for word in freq_words:
    #print(f'{word}\n----------------\n{df[df.lemmatized.str.contains(word)].index.value_counts()}\n')


## leaving off because Ryan and Mark helped me realize what I'm looking into is just TF-IDF and we can easily do this 

# Exploring README name